# Fine-Tuning XLM-RoBERTa для Бинарной Классификации

В этом ноутбуке мы будем настраивать модель `XLM-RoBERTa` для определения, гармонируют ли два текста поста. Мы будем использовать библиотеку `transformers` от Hugging Face и `datasets` для работы с данными.

In [ ]:
# Установка необходимых библиотек
!pip install transformers datasets torch

## Шаг 1: Импорт библиотек и инициализация токенайзера и модели

In [ ]:
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
import torch

# Инициализируем токенайзер и модель
tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")
model = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=2)

## Шаг 2: Подготовка данных

Пример структуры данных: каждый пример состоит из двух текстов и метки (1 — гармонируют, 0 — нет).

In [ ]:
# Пример данных
data = [
    {"text1": "Пример текста поста 1", "text2": "Пример текста поста 2", "label": 1},
    {"text1": "Пример текста поста 3", "text2": "Пример текста поста 4", "label": 0},
    # Добавьте больше данных по необходимости
]

# Преобразуем данные в Dataset
dataset = Dataset.from_list(data)

## Шаг 3: Токенизация данных

Теперь объединим два текста в один вход для модели и токенизируем их.

In [ ]:
# Функция для токенизации
def tokenize_function(examples):
    return tokenizer(
        examples["text1"], examples["text2"], padding="max_length", truncation=True, max_length=128
    )

# Применяем токенизацию
tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")  # Переименовываем колонку для совместимости
tokenized_dataset.set_format("torch")  # Устанавливаем формат для PyTorch

## Шаг 4: Разделение на обучающую и тестовую выборки

Разделим данные на обучающую и тестовую выборки (80% на обучение и 20% на тест).

In [ ]:
# Разделяем на обучающую и тестовую выборки
train_test_split = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

## Шаг 5: Настройка параметров обучения и запуск fine-tuning

In [ ]:
# Настройки для обучения
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Определяем Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Запускаем обучение
trainer.train()

## Шаг 6: Оценка модели

In [ ]:
# Оцениваем модель на тестовой выборке
results = trainer.evaluate()
print("Точность на тестовой выборке:", results["eval_accuracy"])

## Шаг 7: Предсказание для новых текстов

Напишем функцию для предсказания гармоничности двух текстов с использованием обученной модели.

In [ ]:
def predict_harmony(text1, text2):
    inputs = tokenizer(text1, text2, return_tensors="pt", padding="max_length", truncation=True, max_length=128)
    outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=1).item()
    return prediction  # 1 — гармонируют, 0 — нет

# Пример предсказания
text1 = "Пример текста поста 1"
text2 = "Пример текста поста 2"
print("Гармонируют ли посты?", "Да" if predict_harmony(text1, text2) == 1 else "Нет")